In [1]:
import sys
sys.path.append( '../' )

In [2]:
import numpy as np
import cs_vqe as c
import ast
import os

import scipy as sp

In [3]:
from openfermion import qubit_operator_sparse
from quchem.Misc_functions import conversion_scripts as conv_scr
from openfermion.ops import QubitOperator

In [4]:
ham = {
 # noncon
 'ZII':0.5,
 'IXI':0.25,
 'IYI':0.5,
 'IZX':0.5,
 'IZY':0.5,
 'IZZ':0.5,
 'ZXI':0.5,
 'ZYI':0.5,
 'ZZX':0.5,
 'ZZY':0.5,
 'ZZZ':0.25,

 # con
 'IIX':0.5,
 'IIY':0.25,
 'IIZ':0.5,
      }
n_qubits = 3

In [5]:
# ham = {
#  # noncon
#  'ZIIX':0.15,
#  'IXIY':0.25,
#  'IYIZ':0.5,
#  'IZXX':0.5,
#  'IZYY':0.5,
#  'IZZZ':0.5,
#  'ZXIX':0.5,
#  'ZYIX':0.5,
#  'ZZXZ':0.5,
#  'ZZYZ':0.5,
#  'ZZZX':0.25,

#  # con
#  'IIXY':0.5,
#  'IIYY':0.25,
#  'IIZY':0.5,
#       }
# n_qubits = 4

In [6]:
import random

for key in ham.keys():
    ham[key] = random.uniform(-1, 1)*10
ham

{'ZII': -4.876251869647676,
 'IXI': 5.881488492958129,
 'IYI': -1.2892085313386747,
 'IZX': -1.3846171400594853,
 'IZY': -2.987278592805387,
 'IZZ': -1.3947955347513896,
 'ZXI': 1.325160795100755,
 'ZYI': 4.320113689584546,
 'ZZX': -2.5948417951637825,
 'ZZY': -5.389555309824601,
 'ZZZ': 3.5278514245753234,
 'IIX': -7.072699482535942,
 'IIY': 2.683108329914947,
 'IIZ': -7.857755271495231}

In [7]:
# import itertools as it

# # all possible Pauliwords on n_qubits!
# n_qubits = 5

# ham={}
# for pauli_comb in it.product(['X', 'Y', 'Z', 'I'],  repeat=n_qubits):
#     key = ('').join(pauli_comb)
#     ham[key] = random.uniform(5, 10)

# len(ham) == 4**n_qubits

In [8]:
# ham = {
#  # noncon
#  'ZII':0.5,
#  'IXI':0.25,
#  'IYI':0.5,
#  'IZX':0.5,
#  'IZY':0.5,
#  'IZZ':0.5,

#  # con
#  'IIX':0.5,
#  'IIY':0.25,
#  'IIZ':0.5,
#       }
# n_qubits = 3

In [9]:
ham_full_open_F = conv_scr.Get_Openfermion_Hamiltonian(ham)

# Get non-contextual H

In [10]:
nonH_guesses = c.greedy_dfs(ham, 10, criterion='weight')

nonH = max(nonH_guesses, key=lambda x:len(x)) # largest nonCon part found by dfs alg

In [11]:
nonCon_H = {}
Con_H = {}

for P in ham:
    if P in nonH:
        nonCon_H[P]=ham[P]
    else:
        Con_H[P]=ham[P]

        
import pprint

pprint.pprint(Con_H)
print()
pprint.pprint(nonCon_H)

{'IXI': 5.881488492958129,
 'IYI': -1.2892085313386747,
 'ZXI': 1.325160795100755,
 'ZYI': 4.320113689584546}

{'IIX': -7.072699482535942,
 'IIY': 2.683108329914947,
 'IIZ': -7.857755271495231,
 'IZX': -1.3846171400594853,
 'IZY': -2.987278592805387,
 'IZZ': -1.3947955347513896,
 'ZII': -4.876251869647676,
 'ZZX': -2.5948417951637825,
 'ZZY': -5.389555309824601,
 'ZZZ': 3.5278514245753234}


Split into:

$$H = H_{c} + H_{nc}$$

## Testing contextuality

In [12]:
print('Is NONcontextual correct:', not c.contextualQ_ham(nonCon_H))

Is NONcontextual correct: True


# Classical part of problem!

Take $H_{nc}$ and split into:
- $Z$ = operators that completely comute with all operators in $S$
- $T$ = remaining operators in $S$
    - where $S = Z \cup T$  and $S$ is set of Pauli operators in $H_{nc}$
    
    
- We then split the set $T$ into cliques $C_{1}, C_{2}, ... , C_{|T|}$
    - all ops in a clique commute
    - ops between cliques anti-commute!

In [13]:
bool_flag, Z_list, T_list = c.contextualQ(list(nonCon_H.keys()), verbose=True)

In [14]:
Z_list

['ZII']

In [15]:
T_list

['IZX', 'IZY', 'IZZ', 'ZZX', 'ZZY', 'ZZZ', 'IIX', 'IIY', 'IIZ']

## Get quasi model

First we define

- $C_{i1}$ = first Pauli in each $C_{i}$ set
- $A_{ij} = C_{ij}C_{1i}$


- $G^{prime} = \{1 P_{i} \;| \; i=1,2,...,|Z| \}$
    - aka all the completely commuting terms with coefficients set to +1!

- We define G to be an independent set of $G^{prime}$
    - where $G \subseteq G^{prime}$


In [16]:
G_list, Ci1_list, all_mappings = c.quasi_model(nonCon_H)

In [17]:
print('non-independent Z list:', Z_list)
print('G (independent) Z list:', G_list)

non-independent Z list: ['ZII']
G (independent) Z list: ['ZII', 'IZI']


In [18]:
print('all T list (dependent):', T_list)
print('all Ci1 terms (independent):', Ci1_list)

# T list can be built from Ci1 terms AND some combination of G_list!

all T list (dependent): ['IZX', 'IZY', 'IZZ', 'ZZX', 'ZZY', 'ZZZ', 'IIX', 'IIY', 'IIZ']
all Ci1 terms (independent): ['IZZ', 'IZY', 'IZX']


$$R = G \cup \{ C_{i1} \;| \; i=1,2,...,N \}$$

In [19]:
# Assemble all the mappings from terms in the Hamiltonian to their products in R:
all_mappings

{'ZII': [['ZII'], [], 1],
 'IZX': [[], ['IZX'], 1],
 'IZY': [[], ['IZY'], 1],
 'IZZ': [[], ['IZZ'], 1],
 'ZZX': [['ZII'], ['IZX'], 1],
 'ZZY': [['ZII'], ['IZY'], 1],
 'ZZZ': [['ZII'], ['IZZ'], 1],
 'IIX': [['IZI'], ['IZX'], 1],
 'IIY': [['IZI'], ['IZY'], 1],
 'IIZ': [['IZI'], ['IZZ'], 1]}

Overall $R$ is basically reduced non-contextual set
- where everything in original non-contextual set can be found by **inference!**

# Function form

$$R = G \cup \{ C_{i1} \;| \; i=1,2,...,N \}$$

- note q to do with $G$
- note r to do with $C_{i1}$

In [20]:
model = [G_list, Ci1_list, all_mappings]
fn_form = c.energy_function_form(nonCon_H, model)

In [21]:
fn_form

[2,
 3,
 [[-4.876251869647676, [0], [], 'ZII'],
  [-1.3846171400594853, [], [2], 'IZX'],
  [-2.987278592805387, [], [1], 'IZY'],
  [-1.3947955347513896, [], [0], 'IZZ'],
  [-2.5948417951637825, [0], [2], 'ZZX'],
  [-5.389555309824601, [0], [1], 'ZZY'],
  [3.5278514245753234, [0], [0], 'ZZZ'],
  [-7.072699482535942, [1], [2], 'IIX'],
  [2.683108329914947, [1], [1], 'IIY'],
  [-7.857755271495231, [1], [0], 'IIZ']]]

In [22]:
Energy_function = c.energy_function(fn_form)

In [23]:
model = [G_list, Ci1_list, all_mappings]

lowest_eigenvalue, ground_state_params, model_copy, fn_form_copy,  = c.find_gs_noncon(nonCon_H,
               method = 'differential_evolution',
               model=model,
               fn_form=fn_form) # returns:  best + [model, fn_form]

print(lowest_eigenvalue)
print(ground_state_params)

-20.09816919079713
[[1, -1], [-0.6563438070270331, 0.7265801078226785, -0.20320963041560278]]


In [24]:
## check
Energy_function(*ground_state_params[0],*ground_state_params[1]) == lowest_eigenvalue

True

# Now need to rotate Hamiltonian!

We now have non contextual ground state: $(\vec{q}, \vec{r})$

To do this we first rotate each $G_{j}$ and $\mathcal{A} = \sum_{i=1}^{N} r_{i}A_{i}$:

In [25]:
for r_i, A_i in zip(ground_state_params[1],Ci1_list):
    print(r_i, A_i)

-0.6563438070270331 IZZ
0.7265801078226785 IZY
-0.20320963041560278 IZX


In [26]:
model = [G_list, Ci1_list, all_mappings]

print(G_list) # G_j terms!
print(Ci1_list) # mathcal(A)

['ZII', 'IZI']
['IZZ', 'IZY', 'IZX']


to SINGLE QUBIT pauli Z operators!

- to map the operators in $G$ to single qubit Pauli operators, we use $\frac{\pi}{2}$ rotations!

- note $\mathcal{A}$ is an anti-commuting set... therefore we can use $N-1$ rotations as in unitary partitioning's sequence of rotations to do this!
    - $R^{\dagger}\mathcal{A} R = \text{single Pauli op}$

# Rotate full Hamiltonian to basis with diagonal noncontextual generators!

function ```diagonalize_epistemic```:
1. first if else statement:
    - if cliques present:
        - first maps A to single Pauli operator (if cliques present)
        - then rotates to diagonlize G union with single Pauli opator of A (hence GuA name!)
    - else if NO cliques present:
        - gets rotations to diagonlize G
        
     - these rotations make up GuA term in code!
2. NEXT code loops over terms in GuA (denoted as g in code)
    - if g is not a single qubit $Z$:
        - code generates code to rotate operator to make g diagonal (rotations)
        - then constructs map of g to single Z (J rotation)
    - Note R is applied to GuA
    
    
#########
- Note rotations are given in Appendix A of https://arxiv.org/pdf/2011.10027.pdf
    - First code checks if g op in GuA is diagonal
        - if so then needs to apply "K" rotation (involving $Y$ and $I$ operators (see pg 11 top) to make it NOT diagononal
    - now operator will be diagnoal!
    - next generate "J" rotation
        - turns non-diagonal operator into a single qubit $Z$ operator!

In [27]:
# Get sequence of rotations requried to diagonalize the generators for the noncontextual ground state!

Rotations_list, diagonalized_generators_GuA, eigen_vals_nonC_ground_state_GuA_ops = c.diagonalize_epistemic(model,
                                                                                                            fn_form,
                                                                                                            ground_state_params)

In [28]:
script_A_dict = dict(zip(Ci1_list, ground_state_params[1]))

qubit_script_A_list = conv_scr.Get_Openfermion_Hamiltonian(script_A_dict)

qubit_script_A_list

-0.20320963041560278 [Z1 X2] +
0.7265801078226785 [Z1 Y2] +
-0.6563438070270331 [Z1 Z2]

In [29]:
from quchem.Unitary_Partitioning.Unitary_partitioning_Seq_Rot import Get_Xsk_op_list
S_index=0
N_Qubits=n_qubits
check_reduction = True
X_sk_theta_sk_list, normalised_FULL_set, Ps, gamma_l = Get_Xsk_op_list(qubit_script_A_list,
                                                                        S_index,
                                                                        N_Qubits,
                                                                        check_reduction=check_reduction,
                                                                        atol=1e-8,
                                                                        rtol=1e-05)
X_sk_theta_sk_list

R_sk_OP_list = []
for X_sk_Op, theta_sk in X_sk_theta_sk_list:
    op = np.cos(theta_sk / 2) * QubitOperator('') -1j*np.sin(theta_sk / 2) * X_sk_Op
    R_sk_OP_list.append(op)

In [30]:
single_qubit_rot_op_list = []

for ang_op in Rotations_list[len(X_sk_theta_sk_list):]:
    angle = ang_op[0]
    op = ang_op[1]
    theta = np.pi/2
    print(theta, angle)
    P = QubitOperator((' ').join([f'{sigma_i}{q_i}' for q_i, sigma_i in enumerate(op) if sigma_i != 'I']))
    R_op = np.cos(theta / 2) * QubitOperator('') -1j*np.sin(theta / 2) * P
    
    single_qubit_rot_op_list.append(R_op)
single_qubit_rot_op_list

1.5707963267948966 pi/2
1.5707963267948966 pi/2


[0.7071067811865476 [] +
 -0.7071067811865475j [Y2],
 0.7071067811865476 [] +
 -0.7071067811865475j [Z1 Y2]]

In [31]:
from functools import reduce
# print(Rotations_list)

U_dag_op_full_SeqRot = [*R_sk_OP_list, *single_qubit_rot_op_list]
U_dag_op_full_SeqRot_OPERATOR  = reduce(lambda x,y: x*y, U_dag_op_full_SeqRot) 

In [32]:
U_dag_op_full_SeqRot

[0.4059987231605702 [] +
 -0.9138736437779496j [X2],
 0.9947701378322221 [] +
 -0.10213898803719221j [Y2],
 0.7071067811865476 [] +
 -0.7071067811865475j [Y2],
 0.7071067811865476 [] +
 -0.7071067811865475j [Z1 Y2]]

In [33]:
from openfermion import hermitian_conjugated

H_subspace_SEQROT = U_dag_op_full_SeqRot_OPERATOR * ham_full_open_F * hermitian_conjugated(U_dag_op_full_SeqRot_OPERATOR)

len(list(H_subspace_SEQROT))

32

In [34]:
from scipy.linalg import eigh
ham_red_sparse = qubit_operator_sparse(H_subspace_SEQROT, n_qubits=n_qubits)
eig_values_SEQROT, eig_vectors_SEQROT = eigh(ham_red_sparse.todense()) # NOT sparse!
eig_values_SEQROT

array([-25.9149945 , -14.80919378, -14.525861  ,  -0.49741028,
         5.05669005,  10.24991402,  16.16249076,  24.27836473])

In [35]:
H_full_mat = qubit_operator_sparse(ham_full_open_F, n_qubits=n_qubits)
eig_values, eig_vectors = eigh(H_full_mat.todense()) # NOT sparse!
eig_values

array([-25.9149945 , -14.80919378, -14.525861  ,  -0.49741028,
         5.05669005,  10.24991402,  16.16249076,  24.27836473])

In [36]:
np.linalg.norm(eig_values_SEQROT - eig_values)

3.0040920018748386e-14

# NEW LCU method

In [37]:
import quchem.Unitary_Partitioning.Unitary_partitioning_LCU_method as LCU_UP 

N_index=0
N_Qubits=n_qubits
R_linear_comb_list, Pn, gamma_l  = LCU_UP.Get_R_op_list(qubit_script_A_list, 
                                                 N_index, 
                                                 N_Qubits, 
                                                 check_reduction=False, 
                                                 atol=1e-8,
                                                 rtol=1e-05)
Pn

1 [Z1 Z2]

In [38]:
R_linear_comb_list

[0.41452152716895596 [],
 (-0-0.8764081720736903j) [X2],
 (-0-0.24511348277067407j) [Y2]]

In [39]:
import cs_vqe_with_LCU as c_LCU
check_reduction=True
N_Qubits= n_qubits

R_LCU, Rotations_list, diagonalized_generators_GuA, eigen_vals_nonC_ground_state_GuA_ops = c_LCU.diagonalize_epistemic_LCU(
                                      model,
                                      fn_form,
                                      ground_state_params,
                                      check_reduction=True)
R_LCU

[0.41452152716895596 [], -0.8764081720736903j [X2], -0.24511348277067407j [Y2]]

In [40]:
R_LCU_op = reduce(lambda x,y: x+y, R_LCU) 

In [41]:
U_dag_op_full_LCU = [R_LCU_op, *single_qubit_rot_op_list]
U_dag_op_full_LCU_OPERATOR  = reduce(lambda x,y: x*y, U_dag_op_full_LCU) 

In [42]:
H_subspace_LCU = U_dag_op_full_LCU_OPERATOR * ham_full_open_F * hermitian_conjugated(U_dag_op_full_LCU_OPERATOR)
len(list(H_subspace_LCU))

32

In [43]:
ham_red_sparse = qubit_operator_sparse(H_subspace_LCU, n_qubits=n_qubits)
eig_values_LCU, eig_vectors_LCU = eigh(ham_red_sparse.todense()) # NOT sparse!
eig_values_LCU

array([-25.9149945 , -14.80919378, -14.525861  ,  -0.49741028,
         5.05669005,  10.24991402,  16.16249076,  24.27836473])

In [44]:
np.linalg.norm(eig_values_LCU - eig_values)

3.202372833989377e-14

[0.4059987231605702 [] +
 -0.9138736437779496j [X2],
 0.9947701378322221 [] +
 -0.10213898803719221j [Y2]]

In [45]:
print(len(list(H_subspace_LCU)))
print(len(list(H_subspace_SEQROT)))

len(ham)

32
32


14

In [68]:
# X2_Y2
X2_counter=0
Y2_counter=0

X2=QubitOperator('X2')
Y2=QubitOperator('Y2')

for op in ham_full_open_F:
    if op*X2  == -X2*op:
        X2_counter+=1
    if op*Y2  == -Y2*op:
        Y2_counter+=1

len(ham) + X2_counter + (Y2_counter-X2_counter)

20

In [46]:
H_LCU_dict = conv_scr.Openfermion_to_dict(H_subspace_LCU, n_qubits)
H_SEQ_dict = conv_scr.Openfermion_to_dict(H_subspace_SEQROT, n_qubits)

sorted(list(ham.keys()))== sorted(list(H_LCU_dict.keys()))
sorted(list(ham.keys()))== sorted(list(H_SEQ_dict.keys()))

False

In [47]:
# coutner=0
# for P, coeff in H_LCU_dict.items():
#     if abs(coeff.real)> 1e-9:
#         coutner+=1
# coutner

In [48]:
H_LCU = qubit_operator_sparse(H_subspace_LCU, n_qubits=n_qubits).todense()
H_Seq = qubit_operator_sparse(H_subspace_SEQROT, n_qubits=n_qubits).todense()

In [49]:
# np.around(H_LCU-H_Seq, 1)

np.linalg.norm(H_LCU-H_Seq) # <- difference

16.13861254398147

In [50]:
counter=0
for Pword, coeff in H_subspace_SEQROT.terms.items():
    if abs(coeff.real)<1e-9:
        counter+=1
counter

12

In [51]:
counter=0
for Pword, coeff in H_subspace_LCU.terms.items():
    if abs(coeff.real)<1e-9:
        counter+=1
counter

7

In [52]:
len(H_subspace_SEQROT.terms)

32

In [53]:
len(H_subspace_SEQROT.terms.values())

32

In [54]:
print('|A| =', len(script_A_dict))

print()
print('|R_LCU| =', len(list(R_LCU)))

R_S = reduce(lambda x,y: x*y, R_sk_OP_list) 
print('|R_S| =', len(list(R_S)))

|A| = 3

|R_LCU| = 3
|R_S| = 4


size of $R_{LCU} = |A|$ = size of $A$

size of $R_{S}=2^{|A|-1}$

In [55]:
print('|R_S| = 2^{|A|-1}', len(list(R_S)) == 2**(len(script_A_dict)-1))

|R_S| = 2^{|A|-1} True


In [56]:
H_subspace_LCU_Ronly = R_LCU_op * ham_full_open_F * hermitian_conjugated(R_LCU_op)

R_SeqRot  = reduce(lambda x,y: x*y, R_sk_OP_list) 
H_subspace_SEQ_Ronly = R_SeqRot * ham_full_open_F * hermitian_conjugated(R_SeqRot)

In [57]:
counter=0
for Pword, coeff in H_subspace_SEQ_Ronly.terms.items():
    if coeff.real<1e-9:
        counter+=1
print(f'SEQ {counter}')

counter=0
for Pword, coeff in H_subspace_LCU_Ronly.terms.items():
    if coeff.real<1e-9:
        counter+=1
print(f'LCU {counter}')

SEQ 25
LCU 26


In [58]:
ham_CON_open_F = conv_scr.Get_Openfermion_Hamiltonian(Con_H)
Hcon_subspace_LCU_Ronly = R_LCU_op * ham_CON_open_F * hermitian_conjugated(R_LCU_op)
Hcon_subspace_SEQ_Ronly = R_SeqRot * ham_CON_open_F * hermitian_conjugated(R_SeqRot)

In [59]:
counter=0
for Pword, coeff in Hcon_subspace_LCU_Ronly.terms.items():
    if coeff.real<1e-9:
        counter+=1
print(f'SEQ {counter}')
print(len(list(Hcon_subspace_LCU_Ronly)))

counter=0
for Pword, coeff in Hcon_subspace_SEQ_Ronly.terms.items():
    if coeff.real<1e-9:
        counter+=1
print(f'LCU {counter}')
print(len(list(Hcon_subspace_SEQ_Ronly)))

SEQ 13
16
LCU 13
16


In [60]:
print(len(list(ham_full_open_F)))
len(list(ham_CON_open_F))

14


4